# EAST - circulating biomolecules sub-study - CBSS
## mixed effect random forest

In [1]:
# make imports
import warnings
import os
import numpy as np
import pandas as pd
import math

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier

from merf import MERF

import shap

In [2]:
# load data
biomarkers = pd.read_csv('../DATA/input/EAST_biomarker_ML.csv', sep=',', encoding='UTF8')
file = "../DATA/input/east_MI_data_04.dta"
EAST_MI_data = pd.read_stata(file)
EAST_MI_data = EAST_MI_data.loc[EAST_MI_data['_mi_m'] == 0]

In [3]:
# join outcomes
biomarkers = biomarkers.set_index('PID').join(EAST_MI_data[['cid', '_mi_id', '_mi_miss', '_mi_m', 'heart_rhythm_BL', 'heart_rhythm_12', 'heart_rhythm_24', 'subjectnr', 'i_age_calc_10']].\
    set_index('subjectnr'))\
    .rename(columns={'heart_rhythm_BL': 'Cardiac rhythm at baseline',
        'heart_rhythm_12': 'Cardiac rhythm at 12 months follow-up',
                # 'heart_rhythm_24': 'Cardiac rhythm at 24 months follow-up',
                'i_age_calc_10': 'age_10'}).reset_index()
# feature engineering
biomarkers.rename(columns={'age_10': 'Age per 10 years increase'}, inplace=True)
biomarkers['CHA2DS2-Vasc Score'] = biomarkers['CHA2DS2-Vasc Score'].astype('int')
conditions_nyha = [
    (biomarkers['Heart failure (NYHA classification)'] == 'No heart failure'),
    (biomarkers['Heart failure (NYHA classification)'] == 'I'),
    (biomarkers['Heart failure (NYHA classification)'] == 'II'),
    (biomarkers['Heart failure (NYHA classification)'] == 'III')
    ]

values_nyha = ['1', '2', '3', '4']

biomarkers['Heart failure (NYHA classification)'] = np.select(conditions_nyha, values_nyha)

conditions_EHRA = [
    (biomarkers['EHRA score at baseline'] == 'EHRA I'),
    (biomarkers['EHRA score at baseline'] == 'EHRA II'),
    (biomarkers['EHRA score at baseline'] == 'EHRA III'),
    (biomarkers['EHRA score at baseline'] == 'EHRA IV')
    ]

values_EHRA = ['1', '2', '3', '4']

biomarkers['EHRA score at baseline'] = np.select(conditions_EHRA, values_EHRA)

conditions_nights_class = [
    biomarkers['Cognitive function (MoCA) at baseline '] >= 26,
    biomarkers['Cognitive function (MoCA) at baseline '] < 26,
]

values_nights_class = [
    'MoCa >= 26',
    'MoCa < 26']

biomarkers['Cognitive function (MoCA) at baseline category'] = np.select(conditions_nights_class, values_nights_class)

conditions_MoCA = [
    biomarkers['Cognitive function (MoCA) at baseline '] >= 26,
    (biomarkers['Cognitive function (MoCA) at baseline '] >= 18) & (biomarkers['Cognitive function (MoCA) at baseline '] < 26),
    (biomarkers['Cognitive function (MoCA) at baseline '] >= 10) & (biomarkers['Cognitive function (MoCA) at baseline '] < 18),
    biomarkers['Cognitive function (MoCA) at baseline '] < 10,
]

values_MoCA= [
    'None',
    'Mild',
    'Moderate',
    'Severe']

biomarkers['MoCA score'] = np.select(conditions_MoCA, values_MoCA)

biomarkers['At least mild cognitive impairment'] = np.where(biomarkers['MoCA score'] != 'None', 'Yes', 'No')

biomarkers['Cardiac rhythm at baseline binary'] = np.where(biomarkers['Cardiac rhythm at baseline'] == 'Sinus rhythm', 1, 0)
biomarkers['Cardiac rhythm at 24 months follow-up binary'] = np.where(biomarkers['Cardiac rhythm at 24 months follow-up'] == 'Sinus rhythm', 1, 0)
biomarkers['Cardiac rhythm at 12 months follow-up binary'] = np.where(biomarkers['Cardiac rhythm at 12 months follow-up'] == 'Sinus rhythm', 1, 0)
biomarkers['sex'] = np.where(biomarkers['gender'] == 'Male', 'male', 'female')
biomarkers.drop('gender', axis=1, inplace=True)

In [4]:
biomarker = [
'Interleukin-6',
'NT-proBNP',
'Troponin T high sensitive (cTnT-hs)',
'Growth Differentiation Factor-15',
'Cardiac C-Reactive Protein High Sensitive',
'D-Dimer [Explicit units would be "ug FEU/mL]',
'Cancer-Antigen 125 (CA-125)',
'Angiopoietin 2 (ANGPT2)',
'Bone morphogenetic protein 10 (BMP10)',
'Endothelial specific molecule 1 (ESM1)',
'Fatty acid binding protein 3 (FABP3)',
'Fibroblast growth factor 23 (FGF23)',
'Insulin growth factor binding protein 7 (IGFBP7)',
]

In [5]:
# prepare data for rf
y = biomarkers['Cardiac rhythm at 12 months follow-up binary']
X = biomarkers[biomarker + ['Creatinine (enzymatic determination)',
                            'age',
                            'sex',
                            'BMI',
                            'Cardiac rhythm at baseline',
                            'Random group', 'Atrial fibrillation type',
                            'Diastolic blood pressure [mmHg]',
                            'Left ventricular function at Baseline',
                            'cid']]
X = pd.get_dummies(X, drop_first=True)
X['Random group_ERC'] = np.where(X['Random group_Usual care'] == 1, 0, 1)
X.drop('Random group_Usual care', axis=1, inplace=True)

In [ ]:
# fit rf
rfc = RandomForestClassifier()
merf_rfc = MERF(rfc, max_iterations=5)
merf_rfc.fit(X.drop('cid', axis=1), X[['cid']], pd.Series(X['cid']), y)

### Gini feature importance

In [ ]:
sorted_idx = merf_rfc.trained_fe_model.feature_importances_.argsort()
merf_result = pd.DataFrame(merf_rfc.trained_fe_model.feature_importances_).rename(columns={0: 'relative importance in %'})
merf_result['variable'] = X.columns.drop('cid')
merf_result['relative importance in %']  = merf_result['relative importance in %'] * 100

sns.set_theme(style='whitegrid', palette='Set2', context="paper", font_scale=3.0, rc={"lines.linewidth": 2.0})

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(24, 18))
sns.barplot(
    data=merf_result.sort_values(by='relative importance in %', ascending=False), y="variable", x="relative importance in %", color='lightsteelblue', ax=ax
)
for i in np.arange(0.5, 22.5, 2):
    plt.axhspan(i, i+1, facecolor='0.2', alpha=0.2)
ax.set_title("MERF Gini importance (train dataset)")
fig.tight_layout()
# plt.savefig("../DATA/output/merf_gini_importance.svg", bbox_inches="tight")
plt.show()


### SHAP feature importance

In [ ]:
explainer = shap.Explainer(merf_rfc.trained_fe_model.predict, X.drop('cid', axis=1))
shap_values = explainer(X.drop('cid', axis=1))

In [88]:
# give pretty names for plotting
shap_values.feature_names = [name.replace('AF-type_Persistent', 'AF-type: Persistent') for name in shap_values.feature_names]
shap_values.feature_names = [name.replace('AF-type_Paroxysmal', 'AF-type: Paroxysmal') for name in shap_values.feature_names]
shap_values.feature_names = [name.replace('sex_male', 'sex=male') for name in shap_values.feature_names]
shap_values.feature_names = [name.replace('Random group_ERC', 'treatment type: ERC') for name in shap_values.feature_names]
shap_values.feature_names = [name.replace('Cardiac-C-hs', 'CRP') for name in shap_values.feature_names]
shap_values.feature_names = [name.replace('Creatinine (enzymatic determination)', 'sCr') for name in shap_values.feature_names]
shap_values.feature_names = [name.replace('D-Dimer', 'DDimer') for name in shap_values.feature_names]
shap_values.feature_names = [name.replace('TnT-hs', 'TnT') for name in shap_values.feature_names]
shap_values.feature_names = [name.replace('Diastolic blood pressure [mmHg]', 'diastolic BP') for name in shap_values.feature_names]
shap_values.feature_names = [name.replace('Left ventricular function at Baseline', 'LVEF') for name in shap_values.feature_names]
shap_values.feature_names = [name.replace('Cardiac rhythm at baseline_Sinus rhythm', 'Sinus rhythm at BL') for name in shap_values.feature_names]

In [ ]:
sns.set_theme(style='whitegrid')
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.0})

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 12))

g = shap.plots.beeswarm(shap_values, max_display=25, show=False)

fig.suptitle("merf - variable effect by SHAP \n Sinus rhythm at 12 month FU")
fig.tight_layout()
# plt.savefig("../DATA/output/shap_merf.svg", bbox_inches="tight")
plt.show()